In [2]:
import pickle
import numpy as np
import pandas as pd
import string
from tqdm import tqdm

In [3]:
with open('dependencies_new.txt') as file:
    dependencies = pickle.load(file)
with open('dependencies_new_new.txt') as file:
    dependencies_2=pickle.load(file)
data=dependencies+dependencies_2

In [102]:
data=[d for d in data if d != [] and d is not None]

In [82]:
data_new = []
for review in data:
    dependencies = []
    for dependency in review[0]:
        dependency_new = []
        for element in dependency:
            if type(element) == tuple:
                dependency_new.append(list(element))
            else:
                dependency_new.append(element)
        dependencies.append(dependency_new)
    data_new.append([dependencies, review[1]])
with tqdm(total=len(data_new)) as pbar:
    for i in range(len(data_new)):
        for j in range(len(data_new[i][0])):
            data_new[i][0][j][0][0] = data_new[i][0][j][0][0].lower()
            data_new[i][0][j][2][0] = data_new[i][0][j][2][0].lower()
            for punc in string.punctuation:
                if punc in data_new[i][0][j][0][0]:
                    data_new[i][0][j][0][0] = 'N/A'
                if punc in data_new[i][0][j][2][0]:
                    data_new[i][0][j][2][0] = 'N/A'
        pbar.update(1)

In [84]:
total = []
for dependency in data_new:
    if dependency != []:
        if dependency[0] != []:
            total.append(dependency[0])

In [18]:
d={}
for review in total:
    for triple in review:
        triple[0] = tuple(triple[0])
        triple[2] = tuple(triple[2])
        triple = tuple(triple)
        if d.has_key(triple):
            d[triple] += 1
        else:
            d[triple] = 1
d_sorted = sorted(d.items(), key=lambda x:x[1],reverse=True)

In [85]:
features = {}
for review in total:
    for dd in review:
        if 'N/A' in dd[0][0] or 'N/A' in dd[2][0]:
            continue
        if u'JJ' in dd[0][1] and u'NN' in dd[2][1]:
            if features.has_key(dd[2][0]):
                features[dd[2][0]]['count'] += 1
                if dd[0][0] not in features[dd[2][0]]['value']:
                    features[dd[2][0]]['value'].append(dd[0][0])
            else:
                features[dd[2][0]]={}
                features[dd[2][0]]['count']=1
                features[dd[2][0]]['value']=[dd[0][0]]
        elif u'JJ' in dd[2][1] and u'NN' in dd[0][1]:
            if features.has_key(dd[0][0]):
                features[dd[0][0]]['count'] += 1
                if dd[2][0] not in features[dd[0][0]]['value']:
                    features[dd[0][0]]['value'].append(dd[2][0])
            else:
                features[dd[0][0]]={}
                features[dd[0][0]]['count']=1
                features[dd[0][0]]['value']=[dd[2][0]]
features_sorted = sorted(features.items(), key=lambda x:x[1]['count'],reverse=True)

In [126]:
feature_dict = dict(features_sorted[:1000])
for i, key in enumerate(feature_dict.keys()):
    feature_dict[key]['index'] = i
# generate feature matrix
feature_matrix = []
with tqdm(total=len(data)) as pbar:
    for review in data_new:
        feature_vector = np.zeros(1000)
        for dependency in review[0]:
            word1 = dependency[0][0].lower()
            word2 = dependency[2][0].lower()
            if word1 in feature_dict.keys():
                if word2 in feature_dict[word1]['value']:
                    feature_vector[feature_dict[word1]['index']] = feature_dict[word1]['value'].index(word2)
            if word2 in feature_dict.keys():
                if word1 in feature_dict[word2]['value']:
                    feature_vector[feature_dict[word2]['index']] = feature_dict[word2]['value'].index(word1)
        feature_matrix.append(feature_vector,)
        pbar.update(1)
feature_matrix = np.array(feature_matrix)

100%|██████████| 4897/4897 [00:52<00:00, 92.67it/s]


In [129]:
Cell_Phones_and_Accessories = pd.read_json('./data/reviews_Cell_Phones_and_Accessories_5.json', lines=True)
ratings = []
for review in data_new:
    ratings.append(Cell_Phones_and_Accessories.iloc[review[1],2])
np.savetxt('ratings_for_dp.csv', ratings, delimiter=',')

4


In [131]:
np.savetxt('matrix_for_dp.csv', feature_matrix, delimiter=',')

In [135]:
count = 0
new_matrix = []
new_ratings = []
for i in range(len(feature_matrix)):
    if np.sum(feature_matrix[i])!=0:
        new_matrix.append(feature_matrix[i])
        new_ratings.append(ratings[i])

np.savetxt('ratings_for_dp.csv', new_ratings, delimiter=',')
np.savetxt('matrix_for_dp.csv', new_matrix, delimiter=',')

In [136]:
len(new_matrix)

4675